In [3]:
import pandas as pd  # Biblioteca usada para manipulação e análise de dados tabulares, como DataFrame.
from sklearn.feature_extraction.text import TfidfVectorizer  # Para criar uma matriz TF-IDF a partir do texto.
from sklearn.metrics.pairwise import cosine_similarity  # Utilizado para calcular a similaridade de cosseno entre vetores.
from rich.console import Console  # Para exibir mensagens formatadas no console de forma mais estilizada.
from rich.table import Table  # Ajuda a criar tabelas bem formatadas no console.

# Carregar o arquivo CSV
df = pd.read_csv("/content/TMDB_movie_dataset_v11.csv")

# Pré-processamento dos dados
# Selecionar colunas relevantes e remover linhas com valores ausentes em 'overview'
df = df[['title', 'overview', 'genres', 'keywords']].copy()
df['genres'].fillna('', inplace=True)
df['keywords'].fillna('', inplace=True)

# Combinar 'overview', 'genres' e 'keywords' em uma única coluna
df['combined_features'] = df['overview'] + ' ' + df['genres'] + ' ' + df['keywords']

# Substituir NaN na coluna combinada por uma string vazia
df['combined_features'].fillna('', inplace=True)

# Criar uma matriz TF-IDF para a coluna combinada
tfidf = TfidfVectorizer(stop_words='english')  # Stop words são palavras comuns que geralmente não contribuem para o significado de uma frase.
tfidf_matrix = tfidf.fit_transform(df['combined_features'])

# Função para obter recomendações com base na similaridade de cosseno
def obter_recomendacoes(title, tfidf_matrix=tfidf_matrix, df=df):

    """
    Função para obter recomendações com base na similaridade de cosseno.

    Parâmetros:
        title (str): O título do filme para o qual deseja-se obter recomendações.
        tfidf_matrix (sparse matrix, optional): A matriz TF-IDF dos filmes. Padrão é a matriz criada anteriormente.
        df (DataFrame, optional): O DataFrame contendo os dados dos filmes. Padrão é o DataFrame carregado anteriormente.

    Returna:
        DataFrame: Um DataFrame contendo os filmes recomendados com base na similaridade de cosseno com o filme fornecido.
    """

    # Encontrar o índice do filme com o título correspondente
    idx = df.index[df['title'] == title].tolist()[0]

    # Calcular a similaridade de cosseno entre o filme selecionado e todos os outros filmes
    cosine_similarities = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()

    # Obter os índices dos filmes mais similares
    similar_movies_indices = cosine_similarities.argsort()[::-1][1:11]

    # Retornar os filmes recomendados
    return df.iloc[similar_movies_indices].copy()


In [4]:
def escolher_filme(filme_escolhido):

    """
    Função para recomendar filmes com base no filme escolhido.

    Parâmetro:
        filme_escolhido (str): O título do filme escolhido.

    Returna:
        None
    """

    # Obter recomendações de filmes com base no filme escolhido
    recommendations = obter_recomendacoes(filme_escolhido)

    # Exibir os filmes recomendados em uma tabela formatada
    console = Console()

    table = Table(title=f'Filmes recomendados com base no filme "{filme_escolhido}"')
    table.add_column("Título", style="cyan")  # Adicionar coluna para os títulos dos filmes, com estilo ciano.
    table.add_column("Gêneros", style="green")  # Adicionar coluna para os gêneros dos filmes, com estilo verde.

    # Iterar sobre cada filme recomendado e adicionar à tabela
    for index, row in recommendations.iterrows():
        table.add_row(row['title'], row['genres'])

    # Imprimir a tabela no console
    console.print(table)

In [5]:
# obter recomendações para um filme específico
filme_escolhido = 'Edge of Tomorrow'
escolher_filme(filme_escolhido)

         Filmes recomendados com base no filme "Edge of Tomorrow"         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Título                            ┃ Gêneros                            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Aliens Night                      │ Horror, Science Fiction            │
│ Rakka                             │ Action, Science Fiction, Horror    │
│ Alien Invasion 2                  │ Comedy                             │
│ H.G. Wells' The War of the Worlds │ Thriller, Science Fiction          │
│ There is an Alien Here            │ Science Fiction, Comedy            │
│ Planets Around Us                 │ Science Fiction                    │
│ The 5th Wave                      │ Science Fiction, Adventure, Action │
│ Swiss Made                        │ Documentary, Science Fiction       │
│ Forbidden Sky                     │ Science Fiction, Horror            │
│ Alien Investigations              │                                    │
└───────────────────────────────────┴────────────────────────────────────┘